In [1]:
import os

In [2]:
username = os.environ.get("USER") or os.environ.get("USERNAME")
if username:
    print(f"Username: {username}")
else:
    print("Username not found in environment variables.")
    sys.exit()

Username: yun79


In [3]:
from pathlib import Path
cwd = str(Path.cwd())
print(f"PWD: {cwd}")
os.environ["PYTHONPATH"] += f":{cwd}"
os.environ["X509_USER_PROXY"] = f"{cwd}/voms_proxy.txt"
os.environ['XRD_REQUESTTIMEOUT'] = "2400"
!echo $PYTHONPATH

PWD: /work/users/yun79/Run3/copperheadV2
:/work/users/yun79/Run3/copperheadV2


In [4]:
from dask_gateway import Gateway
# gateway = Gateway()
gateway = Gateway(
    "http://dask-gateway-k8s.geddes.rcac.purdue.edu/",
    proxy_address="traefik-dask-gateway-k8s.cms.geddes.rcac.purdue.edu:8786",
)

In [5]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

In [6]:
cluster = gateway.new_cluster(
    # conda_env = "/depot/cms/kernels/coffea2024",
    # conda_env = "/depot/cms/kernels/root632",
    # conda_env = "/depot/cms/kernels/coffea_latest",
    conda_env = "/depot/cms/users/yun79/conda_envs/yun_coffea_latest",
    worker_cores = 1, #2
    worker_memory = 6, #6
    env = dict(os.environ),
)

cluster.adapt(minimum=3, maximum=499) # 399 for low resources, otherwise 199
cluster
# cluster.adapt(100) 

/depot/cms/kernels/coffea_latest/lib/python3.12/site-packages/dask_gateway/client.py:702: GatewayWarning: Adapt with `maximum=499, minimum=3` workers would exceed resource limit of 88 workers. Using `maximum=88, minimum=3` instead.
  warnings.warn(GatewayWarning(msg["msg"]))


In [ ]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

In [7]:
"""
To shutdown the cluster, run this cell
Equivalently, you can also do "squeue -u <username>" 
if this is the only slurm job you're doing
"""

for cluster_info in gateway.list_clusters():
    gateway.connect(cluster_info.name).shutdown()

In [ ]:
for cluster_info in gateway.list_clusters():
    print(cluster_info)

In [ ]:
options = gateway.cluster_options()  

In [ ]:
# options.worker_cores
print(options)

In [ ]:
! squeue -u shar1172